# Week 1 - Exercice 3

Rework outparsers to smaller tasks then review LCEL and RetryWithErrorOutputParser

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers.retry import RetryWithErrorOutputParser, NAIVE_RETRY_WITH_ERROR_PROMPT
from langchain_core.runnables import RunnableParallel, RunnableLambda

import re

In [2]:
import sys

sys.path.append("../")

import agents.technical_analyst.technical_analysis_pydantic_model as _pydantic_models

In [3]:
MODEL = "cogito:8b"
# MODEL = "adeelahmad/ReasonableLLAMA-Jr-3b:latest"
DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine.\n\n"
invocation = {"stock": "APPLE", "timeframe": "5 minutes"}

In [4]:
RETRY_PROMPT_TEMPLATE = NAIVE_RETRY_WITH_ERROR_PROMPT.template.split("\n")
RETRY_PROMPT_TEMPLATE = RETRY_PROMPT_TEMPLATE[:-1] + ["YOU MUST RESPECT THE SCHEMA PROVIDED IN THE PROMPT."] + RETRY_PROMPT_TEMPLATE[-1:]
RETRY_PROMPT_TEMPLATE = DEEP_THINKING_INSTRUCTION + "\n".join(RETRY_PROMPT_TEMPLATE)
RETRY_PROMPT_TEMPLATE

'Enable deep thinking subroutine.\n\nPrompt:\n{prompt}\nCompletion:\n{completion}\n\nAbove, the Completion did not satisfy the constraints given in the Prompt.\nDetails: {error}\nYOU MUST RESPECT THE SCHEMA PROVIDED IN THE PROMPT.\nPlease try again:'

In [5]:
model = ChatOllama(model=MODEL, num_gpu=256, num_ctx=4092 * 0.5, num_predict=1000 * 1)  # , extract_reasoning=True)
error_model = ChatOllama(model=MODEL, num_gpu=256, num_ctx=4092 * 2, num_predict=1000 * 2)

In [6]:
system_template = DEEP_THINKING_INSTRUCTION + ("As trading technical analyst expert, your task is to generate the technical analysis report for {stock} at the specified JSON format. "
                   "Use the data (simulated for the exercice) to generate the technical analysis of {stock} action totally filling the JSON schema described below. "
                   "The timeframe of {stock} data is {timeframe}. "

                   "The format of your response is CRITICAL and MUST ADHERE EXACTLY to the JSON schema described here:"
                   "\n{format_instructions}\n"
                   "Thus, You MUST RESPECT the type of JSON schema entries. "
                   "Once again, the JSON schema described above is CRITICAL and MUST BE RESPECTED."
                )

### SUPPORTEvaluation

In [7]:
support_evaluation_parser = PydanticOutputParser(pydantic_object=_pydantic_models.SUPPORTEvaluation)

support_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock", "timeframe"],
    partial_variables={"format_instructions": support_evaluation_parser.get_format_instructions()}
)

support_retry_parser = RetryWithErrorOutputParser(
    parser=support_evaluation_parser,
    retry_chain = PromptTemplate.from_template(RETRY_PROMPT_TEMPLATE) | error_model | (lambda response:  response.content),
    max_retries=10,
)

support_completion_chain = support_prompt_template | model | RunnableLambda(lambda response: re.sub(r"<think>[\n\W\w]+</think>", "", response.content))

support_chain = RunnableParallel(
    completion=support_completion_chain, prompt_value=support_prompt_template
) | RunnableLambda(lambda response: support_retry_parser.parse_with_prompt(**response))

support_json = support_chain.invoke(invocation)
support_json

Retry: 1
Completion <think>
Okay, let's take a look at the provided schema. The user is asking for a technical analysis report for APPLE stock in 5-minute intervals. The JSON schema has specific requirements that need to be met.

First, I notice that the "supports_evaluation" field should be a string describing the global evaluation of the support levels. In my previous attempt, this was just a descriptive sentence, which aligns with the type expected here (string).

Next, looking at the "support_interaction_status", it's supposed to be an enum from the $defs section. The possible values are things like PRICE_ABOVE_SUPPORT or TESTING_RESISTANCE. My response had "PRICE_ABOVE_SUPPORT" which is valid.

Then, "support_interaction_implication" must also be one of the enum values listed in the $defs. I used POTENTIAL_BUY_ZONE here, which fits into that list.

Now, for each support level (close, middle, far), they need to have their own evaluation strings and numerical values within objects l

SupportEvaluation(evaluation='Strong support levels identified at key technical points', interaction_status=<SupportResistanceInteractionStatus.PRICE_ABOVE_SUPPORT: 'PRICE_ABOVE_SUPPORT'>, interaction_implication=<SupportResistanceInteractionImplication.POTENTIAL_BUY_ZONE: 'POTENTIAL_BUY_ZONE'>, close_level='145.50', middle_level='144.25', far_level='143.00', raw_tool_data=None)

### RESISTANCEEvaluation

In [8]:
resistance_evaluation_parser = PydanticOutputParser(pydantic_object=_pydantic_models.RESISTANCEEvaluation)

resistance_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock", "timeframe"],
    partial_variables={"format_instructions": resistance_evaluation_parser.get_format_instructions()}
)

resistance_retry_parser = RetryWithErrorOutputParser(
    parser=resistance_evaluation_parser,
    retry_chain = PromptTemplate.from_template(RETRY_PROMPT_TEMPLATE) | error_model | (lambda response:  response.content),
    max_retries=10,
)

resistance_completion_chain = resistance_prompt_template | model | RunnableLambda(lambda response: re.sub(r"<think>[\n\W\w]+</think>", "", response.content))

resistance_chain = RunnableParallel(
    completion=resistance_completion_chain, prompt_value=resistance_prompt_template
) | RunnableLambda(lambda response: resistance_retry_parser.parse_with_prompt(**response))

resistance_json = resistance_chain.invoke(invocation)
resistance_json

ResistanceEvaluation(evaluation='Strong bullish pressure evident near upper resistances, potential short-selling zones forming around closest resistance.', interaction_status=<SupportResistanceInteractionStatus.TESTING_SUPPORT: 'TESTING_SUPPORT'>, interaction_implication=<SupportResistanceInteractionImplication.POTENTIAL_BUY_ZONE: 'POTENTIAL_BUY_ZONE'>, close_level='$162.45 - Possible selling zone for active traders with stop-loss orders near this level.', middle_level='$165.75 - Strong support potential if price stabilizes here, likely to attract buyers on dips.', far_level='$169.15 - Less immediate impact unless strong upward momentum develops, may act as secondary resistance point.', raw_tool_data=ResistanceRawToolData(close_value=162.45, middle_value=165.75, far_value=169.15))

### PRICESEvaluation

In [9]:
prices_evaluation_parser = PydanticOutputParser(pydantic_object=_pydantic_models.PRICESEvaluation)

prices_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock", "timeframe"],
    partial_variables={"format_instructions": prices_evaluation_parser.get_format_instructions()}
)

prices_retry_parser = RetryWithErrorOutputParser(
    parser=prices_evaluation_parser,
    retry_chain = PromptTemplate.from_template(RETRY_PROMPT_TEMPLATE) | error_model | (lambda response:  response.content),
    max_retries=10,
)

prices_completion_chain = prices_prompt_template | model | RunnableLambda(lambda response: re.sub(r"<think>[\n\W\w]+</think>", "", response.content))

prices_chain = RunnableParallel(
    completion=prices_completion_chain, prompt_value=prices_prompt_template
) | RunnableLambda(lambda response: prices_retry_parser.parse_with_prompt(**response))

prices_json = prices_chain.invoke(invocation)
prices_json

PricesEvaluation(trend='BULLISH', trading_action=<TradingActions.BUY: 'BUY'>, primary_trend=<TrendCategories.STRONG_BULLISH: 'STRONG_BULLISH'>, primary_trend_number_of_touches=3, secondary_trend=<TrendCategories.BULLISH: 'BULLISH'>, secondary_trend_number_of_touches=2, minor_trend=<TrendCategories.CONSOLIDATION: 'CONSOLIDATION'>, minor_trend_number_of_touches=1, raw_tool_data=PRICESRawValue(prices_value=145.67), trend_evaluation='Strong upward momentum with high volume', chart_pattern='Ascending Triangle forming with potential breakout above $150', potential_chart_pattern='Head and Shoulders pattern forming on hourly chart', candlestick_pattern='Bullish Harami candlestick indicating possible reversal', potential_candlestick_pattern='Potential Morning Star pattern in development')

### Indicators

In [10]:
indicators_evaluation_parser = PydanticOutputParser(pydantic_object=_pydantic_models.Indicators)

indicators_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock", "timeframe"],
    partial_variables={"format_instructions": indicators_evaluation_parser.get_format_instructions()}
)

indicators_retry_parser = RetryWithErrorOutputParser(
    parser=indicators_evaluation_parser,
    retry_chain = PromptTemplate.from_template(RETRY_PROMPT_TEMPLATE) | error_model | (lambda response:  response.content),
    max_retries=10,
)

indicators_completion_chain = indicators_prompt_template | model | RunnableLambda(lambda response: re.sub(r"<think>[\n\W\w]+</think>", "", response.content))

indicators_chain = RunnableParallel(
    completion=indicators_completion_chain, prompt_value=indicators_prompt_template
) | RunnableLambda(lambda response: indicators_retry_parser.parse_with_prompt(**response))

indicators_json = indicators_chain.invoke(invocation)
indicators_json

Retry: 1
Completion <think>
Okay, so I need to generate a technical analysis report for APPLE stock using the given JSON schema. The timeframe is 5 minutes, and it should include evaluations from RSI, MACD, and Bollinger Bands. Let me break down each section based on the schema.

First, I'll start with the RSI evaluation. Since this is simulated data, I'll come up with some plausible numbers and trends. For example, let's say the current RSI value is around 65, which indicates overbought territory. The trend here would be "STRONG_BEARISH" because prices are touching the minor RSI trend line multiple times.

Next, the MACD evaluation. Let's assume that the short moving average is 20, long moving average is 50, and the signal value is -10. This setup suggests a bearish MACD cross-over pattern. The primary trend here would be "BEARISH" with a few touches on the trend line.

Then there's the Bollinger Bands evaluation. Let's say the current price is around $150. The upper band value might 

Indicators(rsi_evaluation=None, macd_evaluation=None, bollinger_bands_evaluation=None)

### VOLUMESEvaluation

In [11]:
volumes_evaluation_parser = PydanticOutputParser(pydantic_object=_pydantic_models.VOLUMESEvaluation)

volumes_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock", "timeframe"],
    partial_variables={"format_instructions": volumes_evaluation_parser.get_format_instructions()}
)

volumes_retry_parser = RetryWithErrorOutputParser(
    parser=volumes_evaluation_parser,
    retry_chain = PromptTemplate.from_template(RETRY_PROMPT_TEMPLATE) | error_model | (lambda response:  response.content),
    max_retries=10,
)

volumes_completion_chain = volumes_prompt_template | model | RunnableLambda(lambda response: re.sub(r"<think>[\n\W\w]+</think>", "", response.content))

volumes_chain = RunnableParallel(
    completion=volumes_completion_chain, prompt_value=volumes_prompt_template
) | RunnableLambda(lambda response: volumes_retry_parser.parse_with_prompt(**response))

volumes_json = volumes_chain.invoke(invocation)
volumes_json

VolumesEvaluation(trend='STRONG_BULLISH', trading_action=<TradingActions.BUY: 'BUY'>, primary_trend=<TrendCategories.BULLISH: 'BULLISH'>, primary_trend_number_of_touches=12, secondary_trend=<TrendCategories.CONSOLIDATION: 'CONSOLIDATION'>, secondary_trend_number_of_touches=5, minor_trend=<TrendCategories.BEARISH: 'BEARISH'>, minor_trend_number_of_touches=3, raw_tool_data=VOLUMESRawValue(volumes_value=1234567.89), trend_evaluation='The current volume trend is extremely bullish, suggesting strong buying pressure and high liquidity in the market.')

### ShortTimeframeData

In [12]:
global_model = ChatOllama(model=MODEL, num_gpu=256, num_ctx=4092 * 2, num_predict=1000 * 2)  # , extract_reasoning=True)
global_error_model = ChatOllama(model=MODEL, num_gpu=256, num_ctx=4092 * 4, num_predict=1000 * 4)

In [13]:
global_system_template = DEEP_THINKING_INSTRUCTION + ("As trading technical analyst expert, "
                   "your task is to generate the technical analysis report for {stock} at the specified JSON format. "
                   "The JSON of the supports evaluation is provided here:\n{support_json}\n"
                   "The JSON of the resistances evaluation is provided here:\n{resistance_json}\n"
                   "The JSON of the prices evaluation is provided here:\n{prices_json}\n"
                   "The JSON of the indicators evaluation is provided here:\n{indicators_json}\n"
                   "The JSON of the volumes evaluation is provided here:\n{volumes_json}\n"
                   "Use the data (simulated for the exercice) to generate the technical analysis of {stock} action totally filling the JSON schema described below. "
                   "The timeframe of {stock} data is {timeframe}. "

                   "The format of your response is CRITICAL and MUST ADHERE EXACTLY to the JSON schema described here:"
                   "\n{format_instructions}\n"
                   "Thus, You MUST RESPECT the type of JSON schema entries. "
                   "Once again, the JSON schema described above is CRITICAL and MUST BE RESPECTED."
                )

In [ ]:
short_evaluation_parser = PydanticOutputParser(pydantic_object=_pydantic_models.ShortTimeframeData)

short_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=global_system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock", "timeframe", "support_json", "resistance_json", "prices_json", "indicators_json", "volumes_json"],
    partial_variables={"format_instructions": short_evaluation_parser.get_format_instructions()}
)

short_retry_parser = RetryWithErrorOutputParser(
    parser=short_evaluation_parser,
    retry_chain = PromptTemplate.from_template(RETRY_PROMPT_TEMPLATE) | global_error_model | (lambda response:  response.content),
    max_retries=10,
)

short_completion_chain = short_prompt_template | global_model | RunnableLambda(lambda response: re.sub(r"<think>[\n\W\w]+</think>", "", response.content))
short_invocation = {"stock": "APPLE", "timeframe": "5 minutes",
                    "support_json": support_json.model_dump_json() ,
                    "resistance_json": resistance_json.model_dump_json(),
                    "prices_json": prices_json.model_dump_json(),
                    "indicators_json": indicators_json.model_dump_json(),
                    "volumes_json": volumes_json.model_dump_json()
                    }

print("Short invocation", short_invocation)

short_chain = RunnableParallel(
    completion=short_completion_chain, prompt_value=short_prompt_template
) | RunnableLambda(lambda response: short_retry_parser.parse_with_prompt(**response))

short_json = short_chain.invoke(short_invocation)
short_json

Short invocation {'stock': 'APPLE', 'timeframe': '5 minutes', 'support_json': '{"evaluation":"Strong support levels identified at key technical points","interaction_status":"PRICE_ABOVE_SUPPORT","interaction_implication":"POTENTIAL_BUY_ZONE","close_level":"145.50","middle_level":"144.25","far_level":"143.00","raw_tool_data":null}', 'resistance_json': '{"evaluation":"Strong bullish pressure evident near upper resistances, potential short-selling zones forming around closest resistance.","interaction_status":"TESTING_SUPPORT","interaction_implication":"POTENTIAL_BUY_ZONE","close_level":"$162.45 - Possible selling zone for active traders with stop-loss orders near this level.","middle_level":"$165.75 - Strong support potential if price stabilizes here, likely to attract buyers on dips.","far_level":"$169.15 - Less immediate impact unless strong upward momentum develops, may act as secondary resistance point.","raw_tool_data":{"close_value":162.45,"middle_value":165.75,"far_value":169.15}}

ShortTimeframeData(data_timeframe=5, supports_evaluation=SupportEvaluation(evaluation='Strong support levels identified at key technical points', interaction_status=<SupportResistanceInteractionStatus.PRICE_ABOVE_SUPPORT: 'PRICE_ABOVE_SUPPORT'>, interaction_implication=<SupportResistanceInteractionImplication.POTENTIAL_BUY_ZONE: 'POTENTIAL_BUY_ZONE'>, close_level='145.50', middle_level='144.25', far_level='143.00', raw_tool_data=None), resistances_evaluation=ResistanceEvaluation(evaluation='Strong bullish pressure evident near upper resistances, potential short-selling zones forming around closest resistance.', interaction_status=<SupportResistanceInteractionStatus.TESTING_SUPPORT: 'TESTING_SUPPORT'>, interaction_implication=<SupportResistanceInteractionImplication.POTENTIAL_BUY_ZONE: 'POTENTIAL_BUY_ZONE'>, close_level='$162.45 - Possible selling zone for active traders with stop-loss orders near this level.', middle_level='$165.75 - Strong support potential if price stabilizes here, l